In [14]:
# Imports

import pandas as pd
import biogeme.biogeme as bio
import biogeme.database as db
from biogeme import models
from biogeme.expressions import Beta
import numpy as np

In [15]:
# initialising data
df = pd.read_csv('Dataset 1 2024.csv')
df.rename(columns={'Unnamed: 0':'ID','CAR OWNERSHIP':'CAR_OWNERSHIP'}, inplace=True)
db = db.Database("Dataset 1 2024",df)

# Model Estimation

In [16]:
## creating the model parameters

# car alternative specific constants
Beta0_car = Beta('Beta0_car', 0, None, None, 0)
BetaTT_car = Beta('BetaTT_car', 0, None, None, 0)
BetaFuel_car = Beta('BetaFuel_car', 0, None, None, 0)
BetaParking_car = Beta('BetaParking_car', 0, None, None, 0)
BetaCongestion_car = Beta('BetaCongestion_car', 0, None, None, 0)

# public transport alternative specific constants
Beta0_PT = Beta('Beta0_PT', 0, None, None, 0)
BetaTT_PT = Beta('BetaTT_PT', 0, None, None, 0)
BetaFare_PT = Beta('BetaFare_PT', 0, None, None, 0)
BetaAC_PT = Beta('BetaAC_PT', 0, None, None, 0)
BetaAT_PT = Beta('BetaAT_PT', 0, None, None, 0)
BetaWT_PT = Beta('BetaWT_PT', 0, None, None, 0)


# cycling alternative specific constants
Beta0_cycling = Beta('Beta0_cycling', 0, None, None, 0)
BetaTT_cycling = Beta('BetaTT_cycling', 0, None, None, 0)
BetaBikeability_cycling = Beta('BetaBikeability_cycling', 0, None, None, 0)

# walking alternative specific constants
Beta0_walking = Beta('Beta0_walking', 0, None, None, 1)
BetaTT_walking = Beta('BetaTT_walking', 0, None, None, 0)
BetaWalkability_walking = Beta('BetaWalkability_walking', 0, None, None, 0)

# demographic attribute generic parameters
BetaAge = Beta('BetaAge', 0, None, None, 0)
BetaGender = Beta('BetaGender', 0, None, None, 0)
BetaIncome = Beta('BetaIncome', 0, None, None, 0)
BetaCarOwnership = Beta('BetaCarOwnership', 0, None, None, 0)


In [17]:
globals().update(db.variables)

In [18]:
# baseline utility functions
baseline_user = BetaAge * AGE + BetaGender * GENDER + BetaIncome * INCOME + BetaCarOwnership * CAR_OWNERSHIP

V_baseline_car = Beta0_car + BetaTT_car * TT_CAR + BetaFuel_car * FC_CAR + BetaParking_car * PC_CAR + BetaCongestion_car * CC_CAR + baseline_user
V_baseline_PT = Beta0_PT + BetaTT_PT * TT_PT + BetaFare_PT * FARE_PT + BetaAC_PT * AC_PT + BetaAT_PT * AT_PT + BetaWT_PT * WT_PT + baseline_user
V_baseline_cycling = Beta0_cycling + BetaTT_cycling * TT_CYCLE + BetaBikeability_cycling * BIKEABILITY_INDEX + baseline_user
V_baseline_walking = Beta0_walking + BetaTT_walking * TT_WALK + BetaWalkability_walking * WALKABILITY_INDEX + baseline_user

In [19]:
# intervention 1, AC_PT = 0
V_intervention1_PT = Beta0_PT + BetaTT_PT * TT_PT + BetaFare_PT * FARE_PT + BetaAT_PT * AT_PT + BetaWT_PT * WT_PT + baseline_user

# intervention 2, AT_PT*reduction
reductionFactor = 0.5 # change###################################################
V_baseline_PT = Beta0_PT + BetaTT_PT * TT_PT + BetaFare_PT * FARE_PT + BetaAC_PT * AC_PT + BetaAT_PT * AT_PT*reductionFactor + BetaWT_PT * WT_PT + baseline_user


In [20]:
# associate utility functions with the numbering of alternatives
V_base = {4: V_baseline_car, 3: V_baseline_PT, 2: V_baseline_cycling, 1: V_baseline_walking}
V_intervention1 = {4: V_baseline_car, 3: V_intervention1_PT, 2: V_baseline_cycling, 1: V_baseline_walking}
V_intervention2 = {4: V_baseline_car, 3: V_baseline_PT, 2: V_baseline_cycling, 1: V_baseline_walking}
av = {4: 1, 3: 1, 2: 1, 1: 1}

In [21]:
# logit model
logprob_base = models.loglogit(V_base, av, CHOSEN_MODE)
logprob_intervention1 = models.loglogit(V_intervention1, av, CHOSEN_MODE)
logprob_intervention2 = models.loglogit(V_intervention2, av, CHOSEN_MODE)

In [22]:
# initialising the models
baseline = bio.BIOGEME(db, logprob_base)
intervention1 = bio.BIOGEME(db, logprob_intervention1)
intervention2 = bio.BIOGEME(db, logprob_intervention2)

baseline.modelName = "baseline_model"
intervention1.modelName = "intervention1_model"
intervention2.modelName = "intervention2_model"

In [23]:
# baseline model estimation
results_baseline = baseline.estimate()
results_baseline.getEstimatedParameters()

C:\Users\majdy\AppData\Local\Temp\ipykernel_8232\2433739308.py:3: DeprecationWarning: getEstimatedParameters is deprecated; use get_estimated_parameters instead.
  results_baseline.getEstimatedParameters()


,Value,Rob. Std err,Rob. t-test,Rob. p-value
Beta0_PT,3.374033e+00,3.266654e-01,10.328712,0.000000e+00
Beta0_car,3.044842e+00,1.721571e-01,17.686411,0.000000e+00
Beta0_cycling,6.529353e-02,9.854902e-02,0.662549,5.076196e-01
BetaAC_PT,-5.680314e-01,4.443695e-02,-12.782862,0.000000e+00
BetaAT_PT,-1.944741e-01,1.885465e-02,-10.314382,0.000000e+00
BetaAge,1.859843e-13,9.105827e-14,2.042476,4.110439e-02
BetaBikeability_cycling,5.355265e-03,8.614264e-04,6.216741,5.075869e-10
BetaCarOwnership,-5.145980e-19,1.797693e+308,-0.000000,1.000000e+00
BetaCongestion_car,-1.294176e-01,1.293187e-02,-10.007644,0.000000e+00
BetaFare_PT,-1.139838e+00,6.916429e-02,-16.480158,0.000000e+00


In [24]:
# intervention 1 model estimation
results_intervention1 = intervention1.estimate()
results_intervention1.getEstimatedParameters()

C:\Users\majdy\AppData\Local\Temp\ipykernel_8232\2281614421.py:3: DeprecationWarning: getEstimatedParameters is deprecated; use get_estimated_parameters instead.
  results_intervention1.getEstimatedParameters()


,Value,Rob. Std err,Rob. t-test,Rob. p-value
Beta0_PT,1.854762e+00,2.778877e-01,6.674503e+00,2.480705e-11
Beta0_car,3.043642e+00,1.719508e-01,1.770066e+01,0.000000e+00
Beta0_cycling,7.251213e-02,9.828171e-02,7.377989e-01,4.606367e-01
BetaAT_PT,-9.442224e-02,9.106931e-03,-1.036817e+01,0.000000e+00
BetaAge,1.030701e-13,1.797693e+308,5.731161e-322,1.000000e+00
BetaBikeability_cycling,5.305578e-03,8.590452e-04,6.176134e+00,6.569034e-10
BetaCarOwnership,-6.507005e-20,1.797693e+308,-0.000000e+00,1.000000e+00
BetaCongestion_car,-1.288766e-01,1.290514e-02,-9.986456e+00,0.000000e+00
BetaFare_PT,-1.048659e+00,6.175346e-02,-1.698137e+01,0.000000e+00
BetaFuel_car,-5.499305e-01,3.979734e-02,-1.381827e+01,0.000000e+00


In [25]:
# intervention 2 model estimation
results_intervention2 = intervention2.estimate()
results_intervention2.getEstimatedParameters()

C:\Users\majdy\AppData\Local\Temp\ipykernel_8232\2835041707.py:3: DeprecationWarning: getEstimatedParameters is deprecated; use get_estimated_parameters instead.
  results_intervention2.getEstimatedParameters()


,Value,Rob. Std err,Rob. t-test,Rob. p-value
Beta0_PT,3.371765e+00,3.265138e-01,10.326560,0.000000e+00
Beta0_car,3.044014e+00,1.721213e-01,17.685281,0.000000e+00
Beta0_cycling,6.807170e-02,9.854981e-02,0.690734,4.897327e-01
BetaAC_PT,-5.677898e-01,4.442254e-02,-12.781569,0.000000e+00
BetaAT_PT,-1.943795e-01,1.884977e-02,-10.312036,0.000000e+00
BetaAge,-4.885375e-14,3.525018e-14,-1.385915,1.657729e-01
BetaBikeability_cycling,5.343045e-03,8.614151e-04,6.202637,5.552481e-10
BetaCarOwnership,-5.003122e-19,1.797693e+308,-0.000000,1.000000e+00
BetaCongestion_car,-1.293330e-01,1.293036e-02,-10.002276,0.000000e+00
BetaFare_PT,-1.139329e+00,6.911351e-02,-16.484891,0.000000e+00


In [26]:
results_baseline.write_latex()
results_intervention1.write_latex()
results_intervention2.write_latex()

# Analysis

In [ ]:
# mode share predictions per model
